In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import contextily as cx
import geopandas as gpd
import matplotlib.pyplot as plt
import pandas as pd
import pyregeon
import statsmodels.api as sm
from sklearn import linear_model, preprocessing

import focalpy

In [ ]:
study_area_filepath = "data/study-area.gpkg"
stations_filepath = "data/stations.gpkg"

buildings_filepath = "data/buildings.gpkg"
tree_canopy_filepath = "data/tree-canopy.tif"
dem_filepath = "data/dem.tif"

ts_df_filepath = "data/heatwaves-2022.csv"

y_name = "T$_{mean}$"

grid_res = 500

buffer_dists = [50, 100, 250, 500]

# viz
heatmap_kwargs = dict(annot=True, fmt=".2f", cmap="coolwarm", vmin=-1, vmax=1)
figwidth = plt.rcParams["figure.figsize"][0]
figheight = plt.rcParams["figure.figsize"][1]

In [ ]:
stations_gdf = gpd.read_file(stations_filepath).set_index("station_id")
stations_gdf.head()

In [ ]:
ax = stations_gdf.plot(column="source", legend=True)
cx.add_basemap(ax, crs=stations_gdf.crs)

In [ ]:
ts_df = pd.read_csv(ts_df_filepath, index_col="time", parse_dates=["time"])
ts_df.head()

In [ ]:
# station_gdf.plot(ts_df.mean(), legend=True, cmap="coolwarm")
ax = stations_gdf.assign(**{y_name: ts_df.mean()}).plot(
    y_name,
    legend=True,
    cmap="coolwarm",
    edgecolor="k",
    legend_kwds={"label": f"{y_name} $\:$ [$\circ$C]"},
)
cx.add_basemap(ax, crs=stations_gdf.crs)

In [ ]:
building_gdf = gpd.read_file(buildings_filepath).set_index("id")
# add a "volume" column
building_gdf["volume"] = building_gdf["area"] * building_gdf["height"]
building_gdf.head()

In [ ]:
fa = focalpy.FocalAnalysis(
    [building_gdf, tree_canopy_filepath, dem_filepath],
    stations_gdf,
    buffer_dists,
    [
        "compute_vector_features",
        "compute_raster_features",
        "compute_terrain_attributes",
    ],
    feature_col_prefixes=["building", "tree", ""],
    feature_methods_args={
        "compute_vector_features": [{"volume": "sum"}],
        "compute_terrain_attributes": [["slope", "topographic_position_index"]],
    },
    feature_methods_kwargs={
        "compute_raster_features": {"stats": "sum"},
        "compute_terrain_attributes": {"stats": "mean"},
    },
)

In [ ]:
y_ser = ts_df.mean().rename(y_name)
X_df = fa.features_df.fillna(0).assign(elevation=stations_gdf["elevation"])
# rescale the features
X_df = pd.DataFrame(
    preprocessing.StandardScaler().fit_transform(X_df),
    index=X_df.index,
    columns=X_df.columns,
).loc[y_ser.index]
# data = pd.concat([y_ser, X_df], axis="columns")

regr = linear_model.LinearRegression().fit(X_df, y_ser)
ax = plt.scatter(y_ser, regr.predict(X_df))
# ax.annotate(regr.score(X_df, y_ser))

In [ ]:
results = sm.OLS(y_ser, sm.add_constant(X_df)).fit()
print(results.summary())

In [ ]:
focalpy.scale_eval_ser(X_df, y_ser)

In [ ]:
# focalpy.scale_eval_df(X_df, y_ser)
scale_of_effect_features = focalpy.scale_of_effect_features(X_df, y_ser)
scale_of_effect_features

In [ ]:
results = sm.OLS(y_ser, sm.add_constant(X_df[scale_of_effect_features])).fit()
print(results.summary())

In [ ]:
grid_gser = pyregeon.generate_regular_grid_gser(
    gpd.read_file(study_area_filepath)["geometry"], grid_res, geometry_type="point"
)

In [ ]:
# TODO: object-oriented approach to compute features and predict for the grid